# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold

In [2]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
41278,280,1213,5.0
100273,610,55118,4.0
18095,113,3551,4.0
48256,313,196,4.0
47671,307,56003,3.0
...,...,...,...
41230,279,139385,5.0
97628,606,1480,3.5
28196,195,2599,2.0
22133,146,2688,4.0


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
280,2.731964,3.715942,1.745361,2.693665,2.922758,1.251609,4.590217,1.244301,3.140239,1.477989,...,2.513596,0.365808,4.599535,1.843495,3.698043,1.051014,3.222551,2.254831,1.276063,0.176632
610,0.491105,0.272938,1.042511,2.601819,4.030434,2.936858,2.677868,0.657620,2.073176,3.745100,...,2.000560,4.102689,3.291476,2.393953,2.548810,3.468709,1.000594,0.617808,1.363230,3.737098
113,0.111663,4.786409,3.546311,3.482699,1.614478,4.674821,3.962975,1.397996,2.018019,3.839768,...,4.953664,4.236930,1.106087,2.141706,1.403069,1.876257,2.336632,1.065579,1.630752,1.391113
313,2.861048,1.484011,3.283126,1.609631,1.860558,0.779392,0.900831,3.465299,4.878073,4.655056,...,4.601464,2.217627,2.147782,2.147971,4.582387,4.763202,0.434465,4.987516,3.038479,4.287011
307,1.516308,0.876747,2.819654,4.572128,0.647665,3.368933,0.754088,3.403415,1.324298,2.495426,...,1.025312,2.632836,2.777470,1.856998,0.391366,1.202608,0.579672,2.352359,4.986666,0.613534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,0.365174,2.171849,3.899382,4.112518,2.402451,4.759817,2.083036,3.252037,2.304898,4.513394,...,2.559456,2.131402,4.426306,1.845346,3.648519,4.120736,4.776713,0.660921,4.428706,2.146720
120,0.893483,1.909891,4.722508,0.222756,0.769470,1.756890,4.448304,2.975970,0.079348,2.407510,...,4.784387,1.838014,0.889615,3.226915,0.358483,4.737048,0.403174,4.848662,0.841023,0.182610
499,2.395999,0.355618,3.664617,3.185563,0.547177,4.452827,3.106625,4.552466,4.923656,3.739488,...,1.682358,1.224778,4.259830,0.456224,0.992866,4.030630,4.057744,3.781300,2.791739,2.381288
364,1.275979,1.791455,4.946203,1.143627,1.637822,0.560258,1.200877,3.284309,0.597332,3.134837,...,4.642448,3.697107,2.311441,1.127112,1.854765,1.969470,0.324257,1.389722,1.552585,1.515702


In [5]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny

def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [6]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent


alpha = 0.00009 # learning speed
delta = 100 # minimal upgrade for each step
lambd = 0.05 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users

    while(True):
        previous_total_error = total_error
        predicted_ratings = np.dot(model, movies.T)         
        # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        gradient = np.dot(errors, movies)
        # znow iloczyn skalarny - tym razem bledow

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
        # pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = users.iloc[:, 1:]
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

        # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        model = model - alpha * ( gradient + lambd * regularization)
        total_error = np.sum(errors)
        print(total_error)
        progress = abs(previous_total_error - total_error)
        if progress < delta:
            break
            
    return model

prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

537232.2486263856
489915.0734463512
450785.4222654854
417816.6562615593
389582.9065142904
365064.091000108
343517.28755566
324391.2695926642
307269.2524320875
291830.1782783604
277822.25960394496
265044.68559715035
253334.80817965633
242559.0396035268
232606.28959411045
223383.15963906268
214810.3680110408
206820.04821961396
199353.6759939158
192360.45513279797
185796.0433065549
179621.53341954338
173802.6298554245
168308.97538191828
163113.59603897942
158192.439533702
153523.98856028396
149088.93475613632
144869.9021719244
140851.21149961514
137018.67809315785
133359.4381878682
129861.7987870865
126515.10751675277
123309.6394066303
120236.49808215065
117287.52927355061
114455.24489168626
111732.75619968171
109113.71483934153
106592.26066102354
104162.97546317607
101820.84187907586
99561.20675827852
97379.74848272395
95272.44773541938
93235.56130562538
91265.59857050104
89359.30034089155
87513.61979968463
85725.70529606205
83992.88478891455
82312.65175847511
80682.65242744857
79100.674

## Część 3. - ocena jakości algorytmu

In [7]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu

positive_threshold = 4.0
negative_threshold = 2.0

def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
    true_positives = 0
    true_negatives = 0 
    false_positives = 0 
    false_negatives = 0
    for index, row in test_ratings_set.iterrows():
        userId, movieId, rating = row['userId'], row['movieId'], row['rating']
        predicted_rating = predicted_ratings[int(movieId)][int(userId)]
        is_prediction_positive = predicted_rating >= positive_threshold
        is_true_value_positive = rating >= positive_threshold
        is_prediction_negative = predicted_rating <= negative_threshold
        is_true_value_negative = rating <= negative_threshold
        if is_prediction_positive:
            if is_true_value_positive:
                true_positives += 1
            elif is_true_value_negative: 
                false_positives += 1
        elif is_prediction_negative:
            if is_true_value_negative:
                true_negatives += 1
            elif is_true_value_positive: 
                false_negatives += 1
        
    recall = true_positives / (true_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives)
    f1 =  2 * ((precision * recall) / (precision + recall))
    accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)
    
    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [8]:
predicted_ratings = prediction_model.dot(movies.T)
calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold)

{'true_positives': 1269,
 'true_negatives': 142,
 'false_positives': 318,
 'false_negatives': 351,
 'accuracy': 0.6783653846153846,
 'precision': 0.7996219281663516,
 'recall': 0.7833333333333333,
 'f1': 0.7913938260056128}

In [9]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
calculate_stats(test_ratings_set, random_prediction, positive_threshold, negative_threshold)

{'true_positives': 2223,
 'true_negatives': 25,
 'false_positives': 615,
 'false_negatives': 58,
 'accuracy': 0.7695994522423828,
 'precision': 0.7832980972515856,
 'recall': 0.9745725558965366,
 'f1': 0.868529009572182}

## Część 4. - istotność statystyczna

In [10]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
positive_tests_count = 0
results = []
random_results = []

for train, test in KFold(n_splits=n_tests, shuffle=True).split(all_ratings):
    # wygeneruj macierz użytkowników i ocen
    train_set = all_ratings.iloc[train]
    test_set = all_ratings.iloc[test]
    ratings = get_ratings(train_set, movies)
    users_no, users = initialize_users(train_set)
    # wytrenuj model
    prediction_model = calculate_user_preferences(users, movies, ratings, train_set, users_no, movies_no, alpha, delta, lambd)
    predicted_ratings = prediction_model.dot(movies.T)
    # oblicz metryki dla wytrenowanego modelu
    my_model_stats = calculate_stats(test_set, predicted_ratings, positive_threshold, negative_threshold)
    results.append(my_model_stats)
    # oblicz metryki dla modelu losowego
    _, random_model = initialize_users(train_set)
    random_prediction = random_model.dot(movies.T)
    random_stats = calculate_stats(test_set, random_prediction, positive_threshold, negative_threshold)
    random_results.append(random_stats)


444569.0345286742
411191.8739980902
382932.783023996
358664.43636007176
337553.31943851
318976.74753234483
302464.3217078995
287656.449117809
274274.8140457402
262101.24542534922
250962.50467853568
240719.26443723
231258.06687850197
222485.41071357665
214323.36698273532
206706.2992475343
199578.38667406142
192891.7348533981
186604.92007368564
180681.85581469373
175090.9008102795
169804.14982646395
164796.8639210654
160047.00819457296
155534.8731800255
151242.76194394595
147154.72930991816
143256.36281611238
139534.5973939092
135977.5575318708
132574.4220287948
129315.30745725587
126191.16723852011
123193.70383171485
120315.29200894
117548.911556114
114888.08803077563
112326.84044061559
109859.6348935462
107481.3434216515
105187.20730501937
102972.80432304986
100834.01944482386
98767.01853996745
96768.22474989279
94834.29720838845
92962.11184205493
91148.74401627417
89391.45282241484
87687.66682763776
86034.97113069984
84431.09558613453
82873.9040755816
81361.38471925058
79891.640932849

10825.27939562226
10698.646231066492
10573.304561810994
10449.237158150781
10326.427087631822
10204.857708700596
10084.512664514665
9965.37587691074
9847.431540524522
9730.664117058495
9615.058329693244
9500.599157638015
9387.271830816748
9275.061824685445
9163.95485517719
9053.936873771367
8944.994062683174
8837.112830170503
8730.27980595444
8624.481836750501
8519.705981907295
8415.939509149865
8313.169890424508
8211.384797842562
8110.57209972023
8010.719856711819
464883.4736959466
428487.1978123634
397494.0735111179
370786.3991293025
347521.8204556897
327058.81772282434
308903.4416857778
292671.04864164506
278058.65891194995
264824.8682336506
252775.15148088816
241751.03395499082
231622.05069273215
222279.72672645366
213633.03112115915
205604.91278753756
198129.63591354302
191150.71087737888
184619.27212745874
178492.7943312124
172734.06672242863
167310.36625925227
162192.78522491327
157355.6798721483
152776.2147707792
148433.98347598274
144310.690569616
140389.88345124773
136656.724

13988.04391862822
13825.960877777008
13665.658222381988
13507.109863149592
13350.290211310197
13195.174166648449
13041.737105875256
12889.954871330598
12739.80376000639
12591.260512878345
12444.302304537194
12298.906733109266
12155.051810457222
12012.71595265178
11871.877970705897
11732.517061562803
11594.61279932997
11458.145126751382
11323.094346910118
11189.441115154546
11057.16643124079
10926.251631685016
10796.678382318836
10668.428671041529
10541.484800763315
10415.829382533619
10291.445328848718
10168.315847133355
10046.424433391272
9925.754866019119
9806.291199779374
9688.017759927201
9570.919136486807
9454.980178672835
9340.185989452651
9226.521920245083
9113.973565752096
9002.526758919095
8892.167566020451
8782.882281866401
8674.6574251281
8567.47973377721
8461.336160636789
8356.213869040585
8252.100228597283
8148.982811057028
8046.849386277302
7945.687918285359
7845.486561434361
7746.233656651106
448646.846310307
415969.54095620057
387951.41116837517
363647.04794975586
34234

In [11]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n):
        p += math.comb(n, i) * math.pow(p, i) * math.pow(1-p, n - i) 
    return p


p = 0.05
metric = 'precision'
# metric = 'recall'
# metric = 'accuracy'
positive_tests_count =  0
for i, stats in enumerate(results):
    if stats[metric] > random_results[i][metric]:
        positive_tests_count += 1
        
print(positive_tests_count)
if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')

5
We are better than random!
